In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
spark = SparkSession.builder.appName('Bai thuc hanh 4').getOrCreate()

In [3]:
GT = spark.read.csv('GlobalTemperatures.csv', header=True, inferSchema=True, sep=',')
GLTBCountry = spark.read.csv('GlobalLandTemperaturesByCountry.csv', header=True, inferSchema=True, sep=',')
GLTBCity = spark.read.csv('GlobalLandTemperaturesByCity.csv', header=True, inferSchema=True, sep=',')
GLTBMajorCity = spark.read.csv('GlobalLandTemperaturesByMajorCity.csv', header=True, inferSchema=True, sep=',')
GLTBState = spark.read.csv('GlobalLandTemperaturesByState.csv', header=True, inferSchema=True, sep=',')

In [4]:
GT.printSchema()
GLTBCountry.printSchema()
GLTBCity.printSchema()
GLTBMajorCity.printSchema()
GLTBState.printSchema()

root
 |-- dt: date (nullable = true)
 |-- LandAverageTemperature: double (nullable = true)
 |-- LandAverageTemperatureUncertainty: double (nullable = true)
 |-- LandMaxTemperature: double (nullable = true)
 |-- LandMaxTemperatureUncertainty: double (nullable = true)
 |-- LandMinTemperature: double (nullable = true)
 |-- LandMinTemperatureUncertainty: double (nullable = true)
 |-- LandAndOceanAverageTemperature: double (nullable = true)
 |-- LandAndOceanAverageTemperatureUncertainty: double (nullable = true)

root
 |-- dt: date (nullable = true)
 |-- AverageTemperature: double (nullable = true)
 |-- AverageTemperatureUncertainty: double (nullable = true)
 |-- Country: string (nullable = true)

root
 |-- dt: date (nullable = true)
 |-- AverageTemperature: double (nullable = true)
 |-- AverageTemperatureUncertainty: double (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = 

In [5]:
from pyspark.sql.functions import col, year, month, rank
from pyspark.sql.window import Window

In [6]:
# 1. For each month, show the name of states having the highest temperature in The US.
us_states = GLTBState.filter((col('Country') == 'United States') & (year('dt') == 2004))
us_states = us_states.withColumn('month', month('dt'))
window1 = Window.partitionBy('month').orderBy(col('AverageTemperature').desc())
rank_us_states = us_states.withColumn('rank', rank().over(window1))
highest_us_states = rank_us_states.filter(col('rank') == 1).select('month', 'State', 'AverageTemperature')
highest_us_states.show()

+-----+-------+------------------+
|month|  State|AverageTemperature|
+-----+-------+------------------+
|    1| Hawaii|21.833000000000002|
|    2| Hawaii|            22.377|
|    3| Hawaii|            21.551|
|    4| Hawaii|            21.858|
|    5|Florida|24.929000000000002|
|    6|Florida|            27.709|
|    7|Florida|            28.033|
|    8|Florida|27.793000000000006|
|    9|Florida|            27.051|
|   10| Hawaii|            24.596|
|   11| Hawaii|            23.753|
|   12| Hawaii|            22.335|
+-----+-------+------------------+



In [13]:
from pyspark.sql.functions import avg
# 2. Determine the average temperature over the years of each country.
data = GLTBCountry.filter(year('dt') == 2004)
avg_temp_by_country = data.groupBy('Country').agg(avg('AverageTemperature').alias('AverageTemperature'))
avg_temp_by_country_sorted = avg_temp_by_country.orderBy('Country')
avg_temp_by_country_sorted.show()

+-------------------+------------------+
|            Country|AverageTemperature|
+-------------------+------------------+
|        Afghanistan| 15.77091666666667|
|             Africa|24.895916666666665|
|            Albania|13.258166666666666|
|            Algeria|23.915999999999997|
|     American Samoa|27.281166666666667|
|            Andorra|            12.043|
|             Angola|22.572833333333335|
|           Anguilla|            27.334|
|         Antarctica|              NULL|
|Antigua And Barbuda|27.151166666666672|
|          Argentina|15.442166666666667|
|            Armenia| 9.720166666666666|
|              Aruba|            28.506|
|               Asia| 8.488833333333334|
|          Australia|22.286916666666666|
|            Austria| 6.906083333333332|
|         Azerbaijan|           12.6195|
|            Bahamas|          25.84975|
|            Bahrain|          27.15025|
|       Baker Island|          26.07625|
+-------------------+------------------+
only showing top

In [14]:
#3. Determine the cities of China which have the temperature lower than 0.
cities_china = GLTBCity.filter((col('Country') == 'China') & (year('dt') == 2004) & (col('AverageTemperature') < 0))
distinct_cities_china_2004 = cities_china.select('City').distinct()
distinct_cities_china_2004.show()

+----------+
|      City|
+----------+
|    Acheng|
|    Anshan|
|     Anqiu|
|   Baoshan|
|    Baiyin|
|      Anda|
|      Aksu|
|    Anyang|
|     Baoji|
|Badaojiang|
|    Ürümqi|
|  Baicheng|
|     Altay|
|   Baoding|
|     Benxi|
|    Boshan|
|   Binzhou|
|   Beipiao|
|  Cangzhou|
| Changchun|
+----------+
only showing top 20 rows

